In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Unzipping `archive.zip` from drive

In [2]:
!unzip drive/My\ Drive/archive.zip > /dev/null #-d drive/My\ Drive/enron

Combining all the datasets into one directory for final export of model 

In [3]:
%%bash
cat << EOF > s.sh
#!/bin/bash 
mkdir enron/{ham,spam} -p
for i in \$(ls | grep "enron[1-6]")
do
   cp \$i/ham/* enron/ham
   cp \$i/spam/* enron/spam
done
EOF
chmod +x s.sh
./s.sh

In [ ]:
import os
import pandas as pd
import sklearn as sk
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

import nltk
nltk.download("stopwords")
nltk.download('punkt')

In [27]:
# Change the folder accordingly
dir = "enron" 
spam_list = [os.path.join(dir+"/spam",f) for f in os.listdir(dir+"/spam")]
ham_list = [os.path.join(dir+"/ham",f) for f in os.listdir(dir+"/ham")]

allHamData, allSpamData = [], []
for obj in ham_list:
  with open(obj,encoding='latin1') as ip:
    allHamData.append(" ".join(ip.readlines()))

for obj in spam_list:
  with open(obj,encoding='latin1') as ip:
    allSpamData.append(" ".join(ip.readlines()))

allHamData = list(set(allHamData))
allSpamData = list(set(allSpamData))

hamPlusSpamData = allHamData + allSpamData
labels = ["ham"]*len(allHamData) + ["spam"]*len(allSpamData)

df = pd.DataFrame({"email": hamPlusSpamData, "label": labels})

In [ ]:
cv_vec = sk.feature_extraction.text.TfidfVectorizer(tokenizer = nltk.word_tokenize, stop_words = nltk.corpus.stopwords.words("english"))
X = cv_vec.fit_transform(df.email)

label_encoder = sk.preprocessing.LabelEncoder()
y = label_encoder.fit_transform(df.label)

X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=8, test_size=0.2)
model = LinearSVC()
model.fit(X_train,y_train)
result = model.predict(X_test)
print(confusion_matrix(y_test,result))
print("SVM Accuracy: ",accuracy_score(y_test,result)*100,"%")

In [ ]:
# SVM Accuracy of enron1:  98.7987987987988 %
# SVM Accuracy of enron2:  99.14163090128756 %
# SVM Accuracy of enron3:  99.0521327014218 %
# SVM Accuracy of enron4:  98.80341880341881 %
# SVM Accuracy of enron5:  99.21798631476051 %
# SVM Accuracy of enron6:  98.41402337228715 %

## Combining all the datasets
###( preparing for export)

In [ ]:
cv_vec = sk.feature_extraction.text.TfidfVectorizer(tokenizer = nltk.word_tokenize, stop_words = nltk.corpus.stopwords.words("english"))
X = cv_vec.fit_transform(df.email)

label_encoder = sk.preprocessing.LabelEncoder()
y = label_encoder.fit_transform(df.label)

model = LinearSVC()
model.fit(X,y)

In [7]:
# Save Model Using Pickle
import pickle
pickle.dump(model, open('SVM.sav', 'wb'))
pickle.dump(cv_vec,open('vectorizer.pk', 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
fin = pickle.load(open('vectorizer.pk', 'rb'))
text = '''Subject: re : 2 . 882 s - > np np
> deat : sun , 15 dec 91 2 : 25 : 2 est > : michael < mmorse @ vm1 . yorku . ca > > subject : re : 2 . 864 query > > wlodek zadrozny ask " anything interest " > construction " s > np np " . . . second , > much relate : consider construction form > discuss list late reduplication ? > logical sense " john mcnamara name " tautologous thus , > level , indistinguishable " , , here ? " . ' john mcnamara name ' tautologous support those logic-base semantics irrelevant'''
X = fin.transform([text])
result = loaded_model.predict(X)
print(result)